In [1]:
import os
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.error')
from rdkit import DataStructs
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()


# create directory for saving similarity matrices
if not os.path.exists('../output/distance_matrices/set2'):
    os.makedirs('../output/distance_matrices/set2')

    





In [2]:
rna_binders_path = '../data/diversity_picking/ECFP6_set2/rna_binders_ECFP6_set2_diversity_picked.pkl'
rna_non_binders_path = '../data/diversity_picking/ECFP6_set2/rna_non_binders_ECFP6_set2_diversity_picked.pkl'
protein_binders_path = '../data/diversity_picking/ECFP6_set2/protein_binders_ECFP6_set2_diversity_picked.pkl'
protein_non_binders_path = '../data/diversity_picking/ECFP6_set2/protein_non_binders_ECFP6_set2_diversity_picked.pkl'

rna_binders = pd.read_pickle(rna_binders_path)
n_mol = len(rna_binders)


rna_non_binders = pd.read_pickle(rna_non_binders_path).head(n_mol)
protein_binders = pd.read_pickle(protein_binders_path).head(n_mol)
protein_non_binders = pd.read_pickle(protein_non_binders_path).head(n_mol)

In [3]:
# Length of all dataframes
print(len(rna_binders), len(rna_non_binders), len(protein_binders), len(protein_non_binders))

1961 1961 1961 1961


In [4]:

# Combine the dataframes
combined_df6 = pd.concat([rna_binders, rna_non_binders, protein_binders, protein_non_binders], ignore_index=True)

n = combined_df6.shape[0]
ecfp_column = ['ECFP6']
df_list = []

for ecfp_col in tqdm(ecfp_column, desc="Calculating distance matrices"):
    if ecfp_col == 'ECFP6':
        combined_df = combined_df6

    distance_matrix = np.zeros((n, n))

    for i in range(n):
        if i % 100 == 0:
            print(i)
        for j in range(i, n):
            distance_matrix[i, j] = 1 - DataStructs.TanimotoSimilarity(combined_df[ecfp_col][i], combined_df[ecfp_col][j])
            distance_matrix[j, i] = distance_matrix[i, j]

    df = pd.DataFrame({'mol': combined_df['mol'], 'source': combined_df['source'], ecfp_col: distance_matrix.tolist()})
    df_list.append(df)

    file_name = f'../output/distance_matrices/v2/distance_matrix_{ecfp_col}_set2.pkl'
    df.to_pickle(file_name)

# Concatenate the DataFrames for all ECFP columns
combined_df = pd.concat(df_list, axis=1)

Calculating distance matrices:   0%|          | 0/1 [00:00<?, ?it/s]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800


In [5]:
# load pickle
dist_mat_ = pd.read_pickle('../output/distance_matrices/v2/distance_matrix_ECFP6_set2.pkl')
dist_mat_

,mol,source,ECFP6
0,<rdkit.Chem.rdchem.Mol object at 0x7f134bd0a2a0>,robin_b,"[0.0, 0.987012987012987, 0.978494623655914, 0...."
1,<rdkit.Chem.rdchem.Mol object at 0x7f134819f420>,robin_b,"[0.987012987012987, 0.0, 0.9814814814814815, 1..."
2,<rdkit.Chem.rdchem.Mol object at 0x7f134819f6f0>,robin_b,"[0.978494623655914, 0.9814814814814815, 0.0, 1..."
3,<rdkit.Chem.rdchem.Mol object at 0x7f134819efc0>,robin_b,"[0.9736842105263158, 1.0, 1.0, 0.0, 0.98837209..."
4,<rdkit.Chem.rdchem.Mol object at 0x7f134819e110>,robin_b,"[0.9590163934426229, 0.9883720930232558, 0.96,..."
...,...,...,...
7839,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f55490>,zinc_dark_m,"[0.8372093023255813, 0.9655172413793104, 1.0, ..."
7840,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f554e0>,zinc_dark_m,"[0.87, 0.8939393939393939, 0.9772727272727273,..."
7841,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f55530>,zinc_dark_m,"[0.9212598425196851, 0.9895833333333334, 0.903..."
7842,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f55580>,zinc_dark_m,"[0.9375, 0.9871794871794872, 0.956521739130434..."


In [6]:
dist_mat_['source'].unique()
dist_mat_

,mol,source,ECFP6
0,<rdkit.Chem.rdchem.Mol object at 0x7f134bd0a2a0>,robin_b,"[0.0, 0.987012987012987, 0.978494623655914, 0...."
1,<rdkit.Chem.rdchem.Mol object at 0x7f134819f420>,robin_b,"[0.987012987012987, 0.0, 0.9814814814814815, 1..."
2,<rdkit.Chem.rdchem.Mol object at 0x7f134819f6f0>,robin_b,"[0.978494623655914, 0.9814814814814815, 0.0, 1..."
3,<rdkit.Chem.rdchem.Mol object at 0x7f134819efc0>,robin_b,"[0.9736842105263158, 1.0, 1.0, 0.0, 0.98837209..."
4,<rdkit.Chem.rdchem.Mol object at 0x7f134819e110>,robin_b,"[0.9590163934426229, 0.9883720930232558, 0.96,..."
...,...,...,...
7839,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f55490>,zinc_dark_m,"[0.8372093023255813, 0.9655172413793104, 1.0, ..."
7840,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f554e0>,zinc_dark_m,"[0.87, 0.8939393939393939, 0.9772727272727273,..."
7841,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f55530>,zinc_dark_m,"[0.9212598425196851, 0.9895833333333334, 0.903..."
7842,<rdkit.Chem.rdchem.Mol object at 0x7f12a9f55580>,zinc_dark_m,"[0.9375, 0.9871794871794872, 0.956521739130434..."
